In [ ]:
import numpy as np
import pandas as pd

from lhorizon import LHorizon
from lhorizon.constants import LUNAR_RADIUS
from lhorizon.kernels import load_metakernel
from lhorizon.lhorizon_utils import (
    make_raveled_meshgrid
)
from lhorizon.target import Targeter

load_metakernel()
from lhorizon.handlers import list_majorbodies, list_sites

### part 1: sequential targeting

imagine that you are examining a series of Earth-based radiofrequency observations of the Moon taken from Arecibo Observatory in 2019. For each of these observations, you have a precise ra/dec pointing in Arecibo-topocentric ICRF coordinates (earth mean equator and equinox of J2000, often called "J2000" for historical reasons) but would like to determine the intercept point of the telescope beam on the Moon in selenodetic coordinates. Horizons and LHorizon can help you determine this.

In [ ]:
# load example data from a CSV file
pointings = pd.read_csv('target_example_ephemeris.csv')
# to take these data, Arecibo swept its receiver back and forth across the Moon in a zigzag pattern.
# since there are ~14K data points, for simplicity, we'll just examine one of these sweeps.
pointings_2 = pointings.loc[pointings['scan_ix'] == 2].reset_index(drop=True)
# we round this to the nearest hundredth of a second to make it easy to align with Horizons 
# (milliseconds are probably not important in this application)
pointings_2.loc[:, 'time'] = pointings_2['time'].str.slice(None, -4)
pointings_2 

In [ ]:
# these pointings are separated in time regularly, so we can assemble 
# a Horizons query using a range of epochs (which Horizons tends to prefer)
epochs = {
    'start': pointings_2['time'].values[0],
    'stop': pointings_2['time'].values[-1],
    # unitless step value asks Horizons to divide the time range equally
    'step': len(pointings_2.index) - 1
}

# we need to query Horizons about the Moon with coordinate origin of Arecibo.
# for many bodies, you can just use the common body name, but both "Moon" and "Luna"
# are considered ambiguous, and using a name doesn't work for most Earth sites.
# fortunately, lhorizon offers some simple tools for finding Horizons id codes.
# it retrieves these lists directly from Horizons, so these lists should always be up to date.

# first, major bodies, like the Moon:
majorbodies = list_majorbodies()
majorbodies.loc[majorbodies['name'].str.contains("Moon")]

In [ ]:
# next, Earth sites, like Arecibo.
# notes: 
# 1. Horizons typically follows Minor Planet Center observatory codes, but _not_ for radiotelescopes. 
# 2. if Horizons doesn't know a site, you can also specify it using a dict of topocentric coordinates.

sites = list_sites()
sites.loc[sites['Observatory Name'] == "Arecibo"]

In [ ]:
# now we can assemble a LHorizon and query it for a reference lunar ephemeris. depending on 
# Horizons' mood, this might take anywhere from 5 seconds to a couple of minutes.
april_moon = LHorizon(target=301, origin=-1, epochs=epochs)
april_moon.query()

In [ ]:
# this lunar ephemeris can then be used to assemble a Targeter object.
# the Moon is typically treated as almost perfectly spherical, so we can use 
# a simple ray-sphere equation to help us find lunar targets for this observation.

april_moon_targeter = Targeter(april_moon, target_radius=LUNAR_RADIUS)

In [ ]:
# we can now use this Targeter to solve the ray-sphere intersection problem between the Moon and the
# Arecibo boresight pointing.
april_moon_targeter.find_targets(pointings_2)
# note that many of these targets will be NaN-valued -- this is because during a lot of the sweep, 
# the boresight isn't on the moon.
april_moon_targeter.ephemerides['topocentric'].dropna()

In [ ]:
# these intersection points are still expressed in the Arecibo-topocentric 'j2000' frame.
# the Targeter object can now use SPICE functions to translate them to a selenodetic frame --
# in this case, we'll use MOON_PA (note that you may need to provide additional SPICE kernels for 
# some frames, but this one is completely handled by built-in kernels that can be loaded using
# lhorizons' lhorizons.kernel.load_metakernel() function).
april_moon_targeter.transform_targets_to_body_frame("j2000", "MOON_PA")
april_moon_targeter.ephemerides['bodycentric'].dropna()

### part 2: grid targeting

instead of 'long', temporally-separated targets, `lhorizons` can also be used to find 'wide', spatially-separated targets. this can be useful in cases where you have an image of a nearby planetary body with known FoV and boresight and would like to know to high resolution what pixels correspond to what coordinates in a bodycentric frame. (you can of course also assemble your own series of temporally-separated gridded bodycentric coordinates by performing grid targeting repeatedly in sequence).

In [ ]:
sites.iloc[777]

In [ ]:
names = sites['Observatory Name'].copy()

In [ ]:
names.loc[names.str.contains('Goethe')]

In [ ]:
# imagine you have a photograph of the Moon. this photograph has been digitized as a 1024 x 1024 raster,
# and has already been rotated into orientation along the RA-DEC directions in topocentric 'j2000'
# and translated such that its apparent 'boresight' is at the sub-Earth point relative to the photographer.

# this was taken at an unimportant place in the mountains above Palm Springs...
origin = {
    'lon': -116.68,
    'lat': 33.82,
    'elevation': 1.5
}
# ...on an otherwise unremarkable evening in 1978.
instant = "1978-06-12 03:33:33"

june_moon = Targeter(
    LHorizon(
        target=301,
        origin=origin,
        epochs=instant
    ),
    target_radius=LUNAR_RADIUS
)
grid_center = 512
grid_len=1024
fov = 10
ra_center_value = june_moon.ephemerides['body']['ra_app_icrf'].values[0]
dec_center_value = june_moon.ephemerides['body']['dec_app_icrf'].values[0]
ra_axis = np.array([(ix - grid_center)*fov/grid_len for ix in np.arange(grid_len)])
dec_axis = np.array([(ix - grid_center)*fov/grid_len for ix in np.arange(grid_len)])
ra_axis += ra_center_value
dec_axis += dec_center_value
raveled = make_raveled_meshgrid((ra_axis, dec_axis), ("ra", "dec"))


In [ ]:
# the find_target_grid function computes intersections across the entirety of the grid
# at the specific instant expressed in the first Targeter epoch.
june_moon.find_target_grid(raveled)
# transforming each of these millionish coordinates to MOON_PA may take a minute.
june_moon.transform_targets_to_body_frame("j2000", "MOON_PA")
june_moon.ephemerides['bodycentric'].dropna()